In [1]:
import sys
import neal
import greedy
import tabu
import time
import numpy as np

from pathlib import Path
current_path = Path().resolve()
sys.path.append(str(current_path / '../code/'))
from experiment import Experiment
from table import Table
from visualisation import Figure

# Load the data

In [2]:
data_folder = '../data/Travelling Salesman Problem/Small/'

file_names = ['bayg29.npz', 'bays29.npz', 'dantzig42.npz',
             'fri26.npz', 'gr17.npz', 'gr21.npz', 'gr24.npz']

loaded_files = [np.load(data_folder + file) for file in file_names]

obj_qubos = [i['cost_function_qubo'] for i in loaded_files]
obj_constants = [i['cost_function_constant'] for i in loaded_files]
con_qubos = [i['constraint_function_qubo'] for i in loaded_files]
con_constants = [i['constraint_function_constant'] for i in loaded_files]

# Prepare the data

Here we already get the qubos and constraints. So we only need to calculate penalties and get the full QUBOs.

In [3]:
minimisation = True # This is a minimisation problem
QUBOs, penalties = Experiment.data_prep_light(obj_qubos, con_qubos, 'Verma&Lewis check', minimisation)
qubo_sizes = [max(qubo, key=tuple)[0] + 1 for qubo in QUBOs]

# Run experiments

In [4]:
# The number of samples we want the sampler to return
repeats = 20

## Greedy

In [5]:
greedy_sampler = greedy.SteepestDescentSampler()
greedy_runs = Experiment.run_sampler(QUBOs, obj_qubos, obj_constants, con_qubos, con_constants, 
                                     greedy_sampler, repeats, num_reads=180)

100.0 %


## Simulated Annealing

In [6]:
sa_sampler = neal.SimulatedAnnealingSampler()
sa_runs = Experiment.run_sampler(QUBOs, obj_qubos, obj_constants, con_qubos, con_constants, 
                                 sa_sampler, repeats, num_reads=30)

100.0 %


## Tabu

In [7]:
tabu_sampler = tabu.TabuSampler()
tabu_runs = Experiment.run_sampler(QUBOs, obj_qubos, obj_constants, con_qubos, con_constants, 
                                   tabu_sampler, repeats, timeout=4000)

100.0 %


# Record the results

In [8]:
greedy_results = Table.record_results(greedy_runs, qubo_sizes, penalties, repeats, minimisation)
sa_results = Table.record_results(sa_runs, qubo_sizes, penalties, repeats, minimisation)
tabu_results = Table.record_results(tabu_runs, qubo_sizes, penalties, repeats, minimisation)

# Display the first repetition table
rep = 0
Table.display_side_by_side(greedy_results[rep], sa_results[rep], tabu_results[rep], titles=['Greedy', 'SA', 'Tabu'])

,Size,Penalty,Objective Function,Broken Constraints,Energy (minimisation)
0,784,6177,2244,0,2244
1,784,8473,2909,0,2909
2,1681,4939,1186,0,1186
3,625,4723,1359,0,1359
4,256,7665,2440,0,2440
5,400,10843,3808,0,3808
6,529,5064,1751,0,1751
,Size,Penalty,Objective Function,Broken Constraints,Energy (minimisation)
0,784,6177,3780,0,3780
1,784,8473,4704,0,4704


# Explore the results

In [9]:
# Show total energies of all tries in all problems in a single df
energies_greedy = Table.columns_to_table(greedy_results, 'Energy (minimisation)')
energies_sa = Table.columns_to_table(sa_results, 'Energy (minimisation)')
energies_tabu = Table.columns_to_table(tabu_results, 'Energy (minimisation)')

energies_tabu

,Energy (minimisation) 0,Energy (minimisation) 1,Energy (minimisation) 2,Energy (minimisation) 3,Energy (minimisation) 4,Energy (minimisation) 5,Energy (minimisation) 6,Energy (minimisation) 7,Energy (minimisation) 8,Energy (minimisation) 9,Energy (minimisation) 10,Energy (minimisation) 11,Energy (minimisation) 12,Energy (minimisation) 13,Energy (minimisation) 14,Energy (minimisation) 15,Energy (minimisation) 16,Energy (minimisation) 17,Energy (minimisation) 18,Energy (minimisation) 19
0,1815,1706,1864,1789,1773,1849,1836,1825,1835,1722,1807,1850,1826,1716,1819,1772,1846,1821,1830,1771
1,2289,2103,2257,2256,2193,2238,2054,2271,2158,2197,2277,2201,2094,2205,2097,2232,2296,2276,2194,2290
2,755,699,819,768,775,763,699,699,751,767,699,787,699,768,787,699,767,699,782,704
3,1104,1024,1073,1030,1051,1029,1067,1091,1038,1048,1039,1061,1073,1075,1030,1054,1021,1076,1060,1051
4,2097,2151,2170,2167,2151,2149,2194,2164,2149,2171,2194,2168,2126,2167,2147,2172,2131,2167,2151,2164
5,2881,3126,2898,2787,3017,2814,2995,2959,3048,2863,2932,2950,2992,2972,3005,3015,3005,2754,2913,2989
6,1379,1413,1407,1404,1416,1408,1407,1400,1426,1384,1432,1419,1411,1382,1376,1430,1371,1384,1423,1391


In [10]:
# Show number of broken constraints of all tries in all problems in a single df
broken_constraints_greedy = Table.columns_to_table(greedy_results, 'Broken Constraints')
broken_constraints_sa = Table.columns_to_table(sa_results, 'Broken Constraints')
broken_constraints_tabu = Table.columns_to_table(tabu_results, 'Broken Constraints')

broken_constraints_greedy

,Broken Constraints 0,Broken Constraints 1,Broken Constraints 2,Broken Constraints 3,Broken Constraints 4,Broken Constraints 5,Broken Constraints 6,Broken Constraints 7,Broken Constraints 8,Broken Constraints 9,Broken Constraints 10,Broken Constraints 11,Broken Constraints 12,Broken Constraints 13,Broken Constraints 14,Broken Constraints 15,Broken Constraints 16,Broken Constraints 17,Broken Constraints 18,Broken Constraints 19
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [11]:
# Show feasible solutions
feasible_full_greedy = Table.feasibility_table(greedy_results)
feasible_full_sa = Table.feasibility_table(sa_results)
feasible_full_tabu = Table.feasibility_table(tabu_results)

feasible_full_tabu

,Feasible 0,Feasible 1,Feasible 2,Feasible 3,Feasible 4,Feasible 5,Feasible 6,Feasible 7,Feasible 8,Feasible 9,Feasible 10,Feasible 11,Feasible 12,Feasible 13,Feasible 14,Feasible 15,Feasible 16,Feasible 17,Feasible 18,Feasible 19
0,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
1,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
2,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
3,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
4,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
5,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
6,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True


This is the final and the most important table. It displays the number of feasible solutions achieved for every problem, the feasibility rate (feasible_soultions/total_solutions), the mean energy of the solutions and the standard deviation.

In [12]:
# Calculate number of feasible solutions with mean and SD (in all runs)
feasible_greedy = Table.feasibility_statistic(greedy_results)
feasible_sa = Table.feasibility_statistic(sa_results)
feasible_tabu = Table.feasibility_statistic(tabu_results)

# Display the table
Table.display_side_by_side(feasible_greedy, feasible_sa, feasible_tabu, titles=['Greedy', 'SA', 'Tabu'])

,Feasible,Feasibility rate,Energy mean,Energy SD
0,20.0,1.0,2270.800000,84.584184
1,20.0,1.0,2883.850000,68.137844
2,20.0,1.0,1191.850000,50.171574
3,20.0,1.0,1307.200000,43.358481
4,20.0,1.0,2446.350000,84.705853
5,20.0,1.0,3806.200000,210.631032
6,20.0,1.0,1737.450000,69.559119
Total,140.0,7.0,15643.700000,611.148088
Mean,20.0,1.0,2234.814286,87.306870
SD,0.0,0.0,924.856453,56.592038


# Save results

In [14]:
data_folder = '../Data/Produced/Travelling Salesman Problem/Small/'
broken_constraints_greedy.to_pickle(data_folder + 'vlewischeck_tsp_greedy_broken_constraints.pkl')
broken_constraints_sa.to_pickle(data_folder + 'vlewischeck_tsp_sa_broken_constraints.pkl')
broken_constraints_tabu.to_pickle(data_folder + 'vlewischeck_tsp_tabu_broken_constraints.pkl')

feasible_greedy.to_pickle(data_folder + 'vlewischeck_tsp_greedy_feasible.pkl')
feasible_sa.to_pickle(data_folder + 'vlewischeck_tsp_sa_feasible.pkl')
feasible_tabu.to_pickle(data_folder + 'vlewischeck_tsp_tabu_feasible.pkl')